Baixando dependências

In [1]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

Importando dependências para o projeto

In [8]:
import cv2 as cv
import numpy as np
import tkinter as tk

Ajuste de Tela Automático

In [7]:
def obter_resolucao():
    # Cria uma instância da janela raiz do tkinter
    root = tk.Tk()
    
    # Oculta a janela
    root.withdraw()
    
    # Obtém a resolução da tela
    largura_tela = root.winfo_screenwidth()
    altura_tela = root.winfo_screenheight()
    
    return largura_tela, altura_tela

Função de player de vídeo:

In [9]:
def exibir_video(video_path):
    captura = cv.VideoCapture(video_path)

    largura, altura = obter_resolucao()

    if not captura.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    while True:
        ret, frame = captura.read()
        if not ret:
            break

        # Redimensiona o frame para caber na tela do notebook
        frame_redimensionado = cv.resize(frame, (largura, altura))

        cv.imshow("Resultado Final", frame_redimensionado)
        
        if cv.waitKey(30) & 0xFF == ord('q'):  # Fecha a exibição se 'q' for pressionado
            break

    captura.release()
    cv.destroyAllWindows()

Função de Detecção de Movimento com Limiarização

In [10]:
def processar_e_salvar_video(video_path, output_path, codec='XVID', fps=30):
    captura = cv.VideoCapture(video_path)

    # Verifica se o vídeo foi aberto corretamente
    if not captura.isOpened():
        print("Erro ao abrir o vídeo.")
        return False  # Retorna False em caso de falha

    # Obtém as dimensões do vídeo
    largura = int(captura.get(cv.CAP_PROP_FRAME_WIDTH))
    altura = int(captura.get(cv.CAP_PROP_FRAME_HEIGHT))

    # Define o codec e cria o objeto VideoWriter para salvar o vídeo
    fourcc = cv.VideoWriter_fourcc(*codec)  # 'XVID' ou 'MJPG', por exemplo
    escritor = cv.VideoWriter(output_path, fourcc, fps, (largura, altura))

    # Lê o primeiro quadro do vídeo
    ret, frame_anterior = captura.read()
    if not ret:
        print("Não foi possível ler o vídeo.")
        captura.release()
        return False  # Retorna False em caso de falha

    # Converte o primeiro quadro para escala de cinza
    frame_anterior_gray = cv.cvtColor(frame_anterior, cv.COLOR_BGR2GRAY)
    frame_anterior_gray = cv.GaussianBlur(frame_anterior_gray, (21, 21), 0)

    while True:
        # Lê o próximo quadro
        ret, frame_atual = captura.read()
        if not ret:
            break

        # Converte o quadro atual para escala de cinza
        frame_atual_gray = cv.cvtColor(frame_atual, cv.COLOR_BGR2GRAY)
        frame_atual_gray = cv.GaussianBlur(frame_atual_gray, (21, 21), 0)

        # Calcula a diferença absoluta entre o quadro atual e o anterior
        diferenca = cv.absdiff(frame_anterior_gray, frame_atual_gray)

        # Aplica uma limiarização (threshold) para identificar mudanças significativas
        _, limiar = cv.threshold(diferenca, 25, 255, cv.THRESH_BINARY)

        # Cria uma imagem vermelha do mesmo tamanho do frame original
        vermelho = np.zeros_like(frame_atual)
        vermelho[:, :] = [0, 0, 255]  # Define cor vermelha para toda a imagem (BGR)

        # Sobrepõe os pixels vermelhos nas áreas onde houve movimento
        movimento = cv.bitwise_and(vermelho, vermelho, mask=limiar)

        # Combina a imagem original com a sobreposição vermelha
        resultado = cv.addWeighted(frame_atual, 1, movimento, 1, 0)

        # Escreve o frame processado no arquivo de vídeo
        escritor.write(resultado)

        # Atualiza o frame anterior para o próximo loop
        frame_anterior_gray = frame_atual_gray

    # Libera os recursos após o processamento
    captura.release()
    escritor.release()

    return True  # Retorna True se o processamento foi concluído com sucesso

Função Principal

In [13]:
def main():
    video_path = "../videos/horizontal.mp4"
    output_path = "../videos/videoOutput.mp4"

    # Processa e salva o vídeo em MP4
    if processar_e_salvar_video(video_path, output_path, codec='mp4v', fps=60):
        print("Vídeo processado e salvo com sucesso.")
    
        # Exibe o vídeo processado com ajuste à tela do notebook
        exibir_video(output_path)

    else:
        print("Falha ao processar o vídeo.")

if __name__ == "__main__":
    main()

Vídeo processado e salvo com sucesso.
